In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.engine.sequential import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
data_path = 'data/2017-20211231_1day.csv'
data = pd.read_csv(data_path)
data = data[(data['timestamp'] >= '2021-01-01') & (data['timestamp'] <= '2021-12-31')]
data = data.drop(['timestamp'], axis=1)

data.head()

,price_open,price_high,price_low,price_close,volume_traded,MACD,SMA,OBV,RSI,MFI,vader_pos,vader_neg,vader_neu,vader_compound,pos_count,neg_count,neu_count
1218,28999.63,29700.00,28720.00,29402.64,8712.534707,2263.184523,28336.928,290500.7511,63.508098,65.160611,0.128043,0.053880,0.818098,0.241190,55.0,18.0,19.0
1219,29410.77,33333.00,29050.00,32216.53,17856.747570,2487.654380,29372.652,308357.4987,68.043507,70.871064,0.114361,0.040320,0.845340,0.197399,52.0,18.0,27.0
1220,32216.51,34800.00,31977.45,33097.83,14158.134780,2557.957514,30518.218,322515.6335,68.131621,92.761716,0.116538,0.068763,0.814699,0.147772,47.0,26.0,20.0
1221,33068.83,33669.76,27734.00,32005.88,22437.597960,2470.850392,31143.134,300078.0355,65.466684,88.970039,0.102255,0.036814,0.860922,0.231502,58.0,18.0,26.0
1222,32054.23,34500.00,29936.26,34039.05,14763.112520,2633.040591,32152.386,314841.1480,67.628885,94.263797,0.097333,0.044781,0.857906,0.232670,52.0,13.0,31.0


In [3]:
data_features = data[['price_close', 'vader_pos', 'vader_neg']]

data_labels = data['price_close']

scaler_feature = MinMaxScaler(feature_range=(0, 1))
scaler_label = MinMaxScaler(feature_range=(0, 1))

feature = scaler_feature.fit_transform(data_features.to_numpy())
label = scaler_label.fit_transform(data_labels.to_numpy().reshape(-1, 1))

In [4]:
def split_data(f, l, delay):
    feature, label = [], []

    for i in range(len(f) - delay):
        feature.append(f[i: i + delay])
        label.append(l[i + delay-1])
        
    return np.array(feature), np.array(label).squeeze()

window_size = 1

feature, label = split_data(feature, label, window_size)


X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)

(291, 1, 3) (73, 1, 3)


In [5]:
model = \
    keras.Sequential([
        layers.Bidirectional(layers.GRU(50)),
        layers.Dropout(0.2),
        layers.Dense(1, activation='tanh'),
    ])

2022-11-04 16:51:26.707313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:51:26.715026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:51:26.715239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:51:26.716037: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
EPOCHS = 500
BATCH_SIZE = 8

model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=3e-6), metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(X_train, y_train, 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS, 
                shuffle=False,
                )

2022-11-04 16:51:27.273329: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/500


2022-11-04 16:51:29.563101: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


37/37 [==============================] - 3s 2ms/step - loss: 0.3981 - root_mean_squared_error: 0.6309
Epoch 2/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3916 - root_mean_squared_error: 0.6258
Epoch 3/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3967 - root_mean_squared_error: 0.6298
Epoch 4/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3884 - root_mean_squared_error: 0.6232
Epoch 5/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3828 - root_mean_squared_error: 0.6187
Epoch 6/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3798 - root_mean_squared_error: 0.6163
Epoch 7/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3782 - root_mean_squared_error: 0.6149
Epoch 8/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3768 - root_mean_squared_error: 0.6139
Epoch 9/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3742 - root_mean_squared_

In [7]:
prediction = np.array(model.predict(X_test))
pred_prices = scaler_label.inverse_transform(prediction)
true_prices = scaler_label.inverse_transform(y_test.reshape(-1,1))

mse = mean_squared_error(pred_prices, true_prices)
rmse = mean_squared_error(pred_prices, true_prices, squared=False)
mae = mean_absolute_error(pred_prices, true_prices)
mape = mean_absolute_percentage_error(pred_prices, true_prices)

print('mse:\t', mse)
print('rmse:\t', rmse)
print('mae:\t', mae)
print('mape:\t', mape)

mse:	 37120353.474556275
rmse:	 6092.647493049001
mae:	 4932.280539383562
mape:	 0.10545830218147634
